In [1]:
import tensorflow as tf 
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
def augment(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [image.shape[0], image.shape[1]])
  image = (image / 255.0)
  image = tf.image.random_crop(image, size=[image.shape[0], image.shape[1], 3])
  image = tf.image.random_brightness(image, max_delta=0.5)
  return image, label

In [3]:
data = cv.imread("left_click.png")
type(data)

numpy.ndarray

In [4]:
x = augment(data, 0)
type(x[0][0].numpy())

numpy.ndarray

In [7]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

def augment_images(image, num_augmented):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

    # Normalize pixel values to the range [0, 1]
    image = np.array(image) / 255.0
    image = image.reshape((1,) + image.shape)

    augmented_images = []
    for _ in range(num_augmented):
        augmented_image = datagen.flow(image, batch_size=1).next()[0]
        augmented_images.append(augmented_image)

    return augmented_images

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\api\preprocessing\image\__init__.py)

In [ ]:
df = pd.DataFrame(columns=["Images","Label"])
df

In [ ]:
num_augmented = 500
augmented_images = augment_images(data, num_augmented)
for i in range(num_augmented):
    a = augmented_images[i]
    # plt.imshow(a)
    df.loc[len(df.index)] = [a,0]
    # plt.show()

In [6]:
df

NameError: name 'df' is not defined

In [10]:
data.shape 

(572, 646, 3)

In [11]:
model_arch = ["Conv","max","Conv","max","Conv","max","Flatten","DNN"]
filters_size= 10
kernal = (3,3)
img_shape = data.shape
dense_layer = 10

In [12]:
model = tf.keras.Sequential([])
for i in range(10):
    model.add(tf.keras.layers.Conv2D(filters= 10, kernel_size=(3,3), activation = "relu",input_shape = data.shape ))
    model.add(tf.keras.layers.Conv2D(filters= 10, kernel_size=(3,3), activation = "relu",input_shape = data.shape ))
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 570, 644, 10)      280       
                                                                 
 conv2d_1 (Conv2D)           (None, 568, 642, 10)      910       
                                                                 
 conv2d_2 (Conv2D)           (None, 566, 640, 10)      910       
                                                                 
 conv2d_3 (Conv2D)           (None, 564, 638, 10)      910       
                                                                 
 conv2d_4 (Conv2D)           (None, 562, 636, 10)      910       
                                                                 
 conv2d_5 (Conv2D)           (None, 560, 634, 10)      910       
                                                                 
 conv2d_6 (Conv2D)           (None, 558, 632, 10)      9

In [13]:
model = tf.keras.Sequential([])
for i in model_arch:
    if i.lower == "conv":
        model.add(tf.keras.layers.Conv2D(filters= filters_size, kernel_size=kernal, activation = "relu",input_shape = img_shape ))
        print(i)
    elif i.lower == "max":
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))  
        print(i)
    elif i.lower == "flatten":
        model.add(tf.keras.layers.Flatten())
        print(i)
    elif i.lower == "dnn":
        model.add(tf.keras.layers.Dense(dense_layer))
        print(i)
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.